In [1]:
import gym

import numpy as np
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Trajectory(object):
    
    def __init__(self):
        self.trajectory = []
        self.total_return = 0
        self.length = 0
        
    def add(self, state, action, reward, state_prime):
        self.trajectory.append((state, action, reward, state_prime))
        self.total_return += reward
        self.length += 1
        
    def sample_segment(self):
        T = len(self.trajectory)

        t1 = np.random.randint(1, T+1)
        t2 = np.random.randint(t1, T+1)

        state = self.trajectory[t1-1][0]
        action = self.trajectory[t1-1][1]

        d_r = 0.0
        for i in range(t1, t2 + 1):
            d_r += self.trajectory[i-1][2]

        d_h = t2 - t1 + 1.0

        return ((state,d_r,d_h),action)
    
class ReplayBuffer(object):
    
    def __init__(self, max_size, last_few):
        """
        @param last_few: Number of episodes from the end of the replay buffer
        used for sampling exploratory commands.
        """
        self.max_size = max_size
        self.cur_size = 0
        self.buffer = []
        
        self.last_few = last_few
        
    def add(self, trajectory):
        self.buffer.append(trajectory)
        
        self.buffer = sorted(self.buffer, key=lambda x: x.total_return, reverse=True)
        self.buffer = self.buffer[:self.max_size]
        
    def sample(self, batch_size):
        trajectories = np.random.choice(self.buffer, batch_size, replace=True)
        
        segments = []
        
        for t in trajectories:
            segments.append(t.sample_segment())
            
        return segments
    
    def sample_command(self):
        eps = self.buffer[:self.last_few]
        
        dh_0 = np.mean([e.length for e in eps])
        
        m = np.mean([e.total_return for e in eps])
        s = np.std([e.total_return for e in eps])
        
        dr_0 = np.random.uniform(m, m+s)
        
        return dh_0, dr_0
        

### A1-1. Initialize replay buffer with warm-up episodes using random actions

In [47]:
env = gym.make('CartPole-v1')

In [117]:
rb = ReplayBuffer(5000, 100)

avg_rewards = []

for _ in range(5000):
    s = env.reset()
    done = False
    ep_reward = 0.0
    t = Trajectory()
    while not done:
        # env.render()
        s_old = s
        action = env.action_space.sample()
        s, reward, done, info = env.step(action)
        t.add(s_old, action, reward, s)
        ep_reward += reward
    avg_rewards.append(ep_reward)    
    # print(f'Episode reward: {ep_reward}')
    rb.add(t)
    
    
env.close()
print(f"Average Episode Reward: {np.mean(avg_rewards)}")

Average Episode Reward: 22.2368


### A1-2 Initialize a behavior function

In [136]:
class NoamLR(torch.optim.lr_scheduler._LRScheduler):
    """
    Implements the Noam Learning rate schedule. This corresponds to increasing the learning rate
    linearly for the first ``warmup_steps`` training steps, and decreasing it thereafter proportionally
    to the inverse square root of the step number, scaled by the inverse square root of the
    dimensionality of the model. Time will tell if this is just madness or it's actually important.
    Parameters
    ----------
    warmup_steps: ``int``, required.
        The number of steps to linearly increase the learning rate.
    """
    def __init__(self, optimizer, warmup_steps):
        self.warmup_steps = warmup_steps
        super().__init__(optimizer)

    def get_lr(self):
        last_epoch = max(1, self.last_epoch)
        scale = self.warmup_steps ** 0.5 * min(last_epoch ** (-0.5), last_epoch * self.warmup_steps ** (-1.5))
        return [base_lr * scale for base_lr in self.base_lrs]

In [150]:
class Behavior(nn.Module):
    def __init__(self, input_shape, num_actions):
        super(Behavior, self).__init__()
        self.fc1 = nn.Linear(input_shape,512)
        self.fc2 = nn.Linear(512,512)
        self.fc3 = nn.Linear(512,512)
        self.fc4 = nn.Linear(512,512)
        self.fc5 = nn.Linear(512,num_actions)

    def forward(self, x):
        output = F.relu(self.fc1(x))
        output = F.relu(self.fc2(output))
        output = F.relu(self.fc3(output))
        output = F.relu(self.fc4(output))
        output = self.fc5(output)
        return output


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
d = env.observation_space.shape[0]
model = Behavior(input_shape=d+2, num_actions=1).to(device) # env.action_space.n
optimizer = torch.optim.Adam(model.parameters())
lr_scheduler = NoamLR(optimizer, 50000)

loss_object = torch.nn.BCEWithLogitsLoss().to(device) #CrossEntropyLoss().to(device)

In [151]:
batch_size = 1024

In [152]:
### A1-3: while stopping criteria is not reached do:
### A1-4:   Improve the behavior function by training on replay buffer

In [153]:
loss_sum = 0
loss_count = 0

In [ ]:
def to_training(s, dr, dh):
    l = s.tolist()
    l.append(dr)
    l.append(dh)
    return l

def segments_to_training(segments):
    x = []
    y = []
    for (s, dr, dh), action in segments:
        l = to_training(s, dr, dh)
        x.append(l)
        y.append(action)
        
    x = torch.tensor(x).float().to(device)
    y = torch.tensor(y).float().to(device)
    
    return x, y
        
# accuracy_m = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

def train_step(inputs, targets):
    optimizer.zero_grad()
    
    predictions = model(inputs)
    
    #print(predictions, targets)
    
    loss = loss_object(predictions, targets.unsqueeze(1))
    
    loss.backward()
    optimizer.step()
    
    return loss


def generate_episode(cmd):
    s = env.reset()
    done = False
    ep_reward = 0.0
    
    t = Trajectory()
    while not done:
        (dh, dr) = cmd
        inputs = torch.tensor([to_training(s, dr, dh)]).float().to(device)
        
        action_probs = model(inputs)
        action_probs = F.sigmoid(action_probs) #, dim=-1)
        
        m = torch.distributions.bernoulli.Bernoulli(probs=action_probs) #categorical.Categorical(probs=action_probs)
        action = int(m.sample().squeeze().cpu().numpy())
        
        # env.render()
        s_old = s
        
        s, reward, done, info = env.step(action)
        t.add(s_old, action, reward, s)
        
        ep_reward += reward
        dh = dh - 1
        dr = dr - reward
        cmd = (dh, dr)
    
    # print(f'Episode reward: {ep_reward}')
    rb.add(t)
    return ep_reward
    
    
epochs = 1000000

for i in range(1, epochs+1):
    segments = rb.sample(batch_size)
    segments = np.array(segments)
    x, y = segments_to_training(segments)
    loss = train_step(x, y)
    loss_sum += loss
    loss_count += 1
    
    #if i % 1000 == 0:
    lr_scheduler.step()
    #print(lr_scheduler.get_lr())
    
    if i % 1000 == 0:
        rewards = [] 
        for e in range(1000):
            cmd = rb.sample_command()
            rewards.append(generate_episode(cmd))
        
        print(f"Average Episode Reward: {np.mean(rewards)}")
    
    if i % 200 == 0:
        print(f'i: {i}, Loss: {loss_sum/loss_count}') #'\t Accuracy: {accuracy_m.result()}')
    
    

i: 200, Loss: 0.6454012393951416
i: 400, Loss: 0.64532470703125
i: 600, Loss: 0.6452473402023315
i: 800, Loss: 0.6451734900474548
Average Episode Reward: 95.887
i: 1000, Loss: 0.6451003551483154
i: 1200, Loss: 0.6450269222259521
i: 1400, Loss: 0.6449548006057739
i: 1600, Loss: 0.644880473613739
i: 1800, Loss: 0.6448032855987549
Average Episode Reward: 152.697
i: 2000, Loss: 0.6447285413742065
i: 2200, Loss: 0.6446495056152344
i: 2400, Loss: 0.644574761390686
i: 2600, Loss: 0.6445000767707825
i: 2800, Loss: 0.6444293260574341
Average Episode Reward: 108.545
i: 3000, Loss: 0.6443573832511902
i: 3200, Loss: 0.6442849636077881
i: 3400, Loss: 0.6442137360572815
i: 3600, Loss: 0.6441410183906555
i: 3800, Loss: 0.6440678238868713
Average Episode Reward: 144.435
i: 4000, Loss: 0.6439946889877319
i: 4200, Loss: 0.6439268589019775
i: 4400, Loss: 0.6438541412353516
i: 4600, Loss: 0.6437812447547913
i: 4800, Loss: 0.6437135338783264
Average Episode Reward: 144.913
i: 5000, Loss: 0.6436429619789124

In [ ]:
rb.sample_command()

In [91]:
## A1:6 Generate episodes using Alg 2 and add to replay buffer

In [106]:
cmd = (500, 500) #rb.sample_command()

In [155]:
avg_rewards = []

def test(cmd):
    s = env.reset()
    done = False
    ep_reward = 0.0
    
    while not done:
        (dh, dr) = cmd
        inputs = torch.tensor([to_training(s, dr, dh)]).float().to(device)
        
        action_probs = model(inputs)
        action_probs = F.sigmoid(action_probs) #, dim=-1)
        
        m = torch.distributions.bernoulli.Bernoulli(probs=action_probs) #torch.distributions.categorical.Categorical(probs=action_probs)
        action = int(m.sample().squeeze().cpu().numpy())
        
        #env.render()
        s_old = s
        
        s, reward, done, info = env.step(action)
        
        ep_reward += reward
        dh = dh - 1
        dr = dr - reward
        cmd = (dh, dr)
    
    # print(f'Episode reward: {ep_reward}')
    return ep_reward

rewards = [] 
for e in range(100):
    rewards.append(test(cmd))

# env.close()
print(f"Average Episode Reward: {np.mean(rewards)}")

Average Episode Reward: 22.03
